In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
from sklearn import metrics
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from itertools import chain
from scipy.stats import chisquare,chi2_contingency

Using TensorFlow backend.


### Working on Data

In [4]:
df = pd.read_csv("bank.csv")

In [5]:
df.head()
df.tail()
df.shape
df.dtypes
df.info

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


(10000, 14)

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

<bound method DataFrame.info of       RowNumber  CustomerId         Surname  CreditScore Geography  Gender  \
0             1    15634602        Hargrave          619    France  Female   
1             2    15647311            Hill          608     Spain  Female   
2             3    15619304            Onio          502    France  Female   
3             4    15701354            Boni          699    France  Female   
4             5    15737888        Mitchell          850     Spain  Female   
5             6    15574012             Chu          645     Spain    Male   
6             7    15592531        Bartlett          822    France    Male   
7             8    15656148          Obinna          376   Germany  Female   
8             9    15792365              He          501    France    Male   
9            10    15592389              H?          684    France    Male   
10           11    15767821          Bearce          528    France    Male   
11           12    15737173     

In [6]:
df.isnull().values.any()
df.isnull().sum().sum()
df.isna().values.any()
df.isna().sum().sum()
df.describe()
df.describe().transpose()

False

0

False

0

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5.000500e+03,2886.895680,1.00,2500.75,5.000500e+03,7.500250e+03,10000.00
CustomerId,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
CreditScore,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
Age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
Tenure,10000.0,5.012800e+00,2.892174,0.00,3.00,5.000000e+00,7.000000e+00,10.00
Balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
NumOfProducts,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
HasCrCard,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
IsActiveMember,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
EstimatedSalary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48


### Drop the columns which are unique for all users like IDs

In [7]:
#dropping RowNumber, CustomerId, Surname
df.drop(columns = ["RowNumber","CustomerId","Surname"], axis =1, inplace =True)

#### Encoding Catergorical Features

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [9]:
labelencoder= LabelEncoder()

In [10]:
df['Gender']=labelencoder.fit_transform(df['Gender'])

In [11]:
Geography =  pd.get_dummies(df["Geography"])

In [12]:
df = pd.concat([df, Geography],axis=1)

In [13]:
df.drop(columns = ["Geography"], inplace =True, axis = 1)

In [14]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1


### Target and Feature - Train Test Split

In [15]:
X = df.drop(columns = ["Exited"])
y = df["Exited"]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 15,stratify=y)

### Standardizing the data

In [17]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\Users\rajan\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\rajan\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\rajan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int32, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


## Model Building

In [18]:
from keras.models import Sequential
from keras.layers import Dense

In [19]:
classifier = Sequential()

In [20]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 12))
# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
# Adding the third hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\rajan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=12, units=6, kernel_initializer="uniform")`
  
C:\Users\rajan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\rajan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  
C:\Users\rajan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


In [21]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [22]:
classifier.fit(X_train, y_train, batch_size=30, epochs=100)


Instructions for updating:
Use tf.cast instead.
Epoch 1/100
7000/7000 [==============================] - 1s 156us/step - loss: 0.5823 - acc: 0.7946
Epoch 2/100
7000/7000 [==============================] - 0s 28us/step - loss: 0.4305 - acc: 0.7963
Epoch 3/100
7000/7000 [==============================] - 0s 29us/step - loss: 0.4192 - acc: 0.7963
Epoch 4/100
7000/7000 [==============================] - 0s 28us/step - loss: 0.4131 - acc: 0.7963
Epoch 5/100
7000/7000 [==============================] - 0s 29us/step - loss: 0.4069 - acc: 0.7963
Epoch 6/100
7000/7000 [==============================] - 0s 28us/step - loss: 0.4008 - acc: 0.8136
Epoch 7/100
7000/7000 [==============================] - 0s 28us/step - loss: 0.3957 - acc: 0.8296
Epoch 8/100
7000/7000 [==============================] - 0s 27us/step - loss: 0.3916 - acc: 0.8293
Epoch 9/100
7000/7000 [==============================] - 0s 27us/step - loss: 0.3873 - acc: 0.8309
Epoch 10/100
7000/7000 [==============================] - 0s

7000/7000 [==============================] - 0s 27us/step - loss: 0.3449 - acc: 0.8606
Epoch 81/100
7000/7000 [==============================] - 0s 26us/step - loss: 0.3461 - acc: 0.8606
Epoch 82/100
7000/7000 [==============================] - 0s 27us/step - loss: 0.3457 - acc: 0.8587
Epoch 83/100
7000/7000 [==============================] - 0s 28us/step - loss: 0.3466 - acc: 0.8610
Epoch 84/100
7000/7000 [==============================] - 0s 27us/step - loss: 0.3457 - acc: 0.8599
Epoch 85/100
7000/7000 [==============================] - 0s 26us/step - loss: 0.3449 - acc: 0.8596
Epoch 86/100
7000/7000 [==============================] - 0s 27us/step - loss: 0.3461 - acc: 0.8573
Epoch 87/100
7000/7000 [==============================] - 0s 27us/step - loss: 0.3451 - acc: 0.8604
Epoch 88/100
7000/7000 [==============================] - 0s 27us/step - loss: 0.3459 - acc: 0.8584
Epoch 89/100
7000/7000 [==============================] - 0s 27us/step - loss: 0.3448 - acc: 0.8599
Epoch 90/100


## Prediction and Accuracy

In [23]:
y_pred = classifier.predict(X_test)

In [24]:
acc=metrics.accuracy_score(y_test,y_pred > 0.5)
print(acc)

0.86
